In [1]:
import numpy as np
from sklearn.metrics import accuracy_score
import torch
import torch.nn.functional as F
from ogb.nodeproppred import PygNodePropPredDataset, Evaluator
import torch_geometric.transforms as T
from torch_geometric.data import DataLoader
from torch_geometric.nn import SAGEConv

In [2]:
dataset = PygNodePropPredDataset(name='ogbn-arxiv', 
                                 root='../../data/dataset/',
                                 transform=T.ToSparseTensor())

In [3]:
torch.cuda.is_available()

True

In [4]:
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
device = torch.device(device)

In [5]:
data = dataset[0]
data.adj_t = data.adj_t.to_symmetric()

In [6]:
data

Data(adj_t=[169343, 169343, nnz=2315598], node_year=[169343, 1], x=[169343, 128], y=[169343, 1])

In [7]:
data = data.to(device)

In [8]:
split_idx = dataset.get_idx_split()
train_idx = split_idx['train'].to(device)

In [9]:
evaluator = Evaluator(name='ogbn-arxiv')

In [10]:
data['x'].shape[1]

128

In [11]:
data['y'][:, 0].unique().shape[0]

40

In [12]:
n_iters = 3000
epochs = 500
log_steps = 25
input_dim = data['x'].shape[1]
hidden_channels = input_dim * 2
output_dim = data['y'][:, 0].unique().shape[0]
lr_rate = 0.001

In [13]:
class ThreeLayer(torch.nn.Module):
    def __init__(self, input_dim, hidden_channels, output_dim, dropout):
        super(ThreeLayer, self).__init__()
        
        self.convs = torch.nn.ModuleList()
        self.bns = torch.nn.ModuleList()
        
        self.convs.append(SAGEConv(input_dim, hidden_channels))
        self.bns.append(torch.nn.BatchNorm1d(hidden_channels))
        
        self.convs.append(SAGEConv(hidden_channels, hidden_channels))
        self.bns.append(torch.nn.BatchNorm1d(hidden_channels))
        
        self.convs.append(SAGEConv(hidden_channels, hidden_channels))
        self.bns.append(torch.nn.BatchNorm1d(hidden_channels))
        
        self.convs.append(SAGEConv(hidden_channels, output_dim))
        
        self.dropout = dropout
        

    def forward(self, x, adj_t):
        x = self.convs[0](x, adj_t)
        x = self.bns[0](x)
        x = F.relu(x)
        x = F.dropout(x, p=self.dropout, training=self.training)
        
        x = self.convs[1](x, adj_t)
        x = self.bns[1](x)
        x = F.relu(x)
        x = F.dropout(x, p=self.dropout, training=self.training)
        
        x = self.convs[2](x, adj_t)
        x = self.bns[2](x)
        x = F.relu(x)
        x = F.dropout(x, p=self.dropout, training=self.training)
        
        x = self.convs[3](x, adj_t)
        return x.log_softmax(dim=-1)

In [14]:
model = ThreeLayer(input_dim, hidden_channels, output_dim, 0.5).to(device)

In [15]:
def train(model, data, train_idx, optimizer):
    model.train()

    optimizer.zero_grad()
    out = model(data.x, data.adj_t)[train_idx]
    loss = F.nll_loss(out, data.y.squeeze(1)[train_idx])
    loss.backward()
    optimizer.step()

    return loss.item()

In [16]:
@torch.no_grad()
def test(model, data, split_idx, evaluator):
    model.eval()

    out = model(data.x, data.adj_t)
    y_pred = out.argmax(dim=-1, keepdim=True)

    train_acc = evaluator.eval({
        'y_true': data.y[split_idx['train']],
        'y_pred': y_pred[split_idx['train']],
    })['acc']
    valid_acc = evaluator.eval({
        'y_true': data.y[split_idx['valid']],
        'y_pred': y_pred[split_idx['valid']],
    })['acc']
    test_acc = evaluator.eval({
        'y_true': data.y[split_idx['test']],
        'y_pred': y_pred[split_idx['test']],
    })['acc']

    return train_acc, valid_acc, test_acc

In [17]:
optimizer = torch.optim.Adam(model.parameters(), lr=lr_rate)
for epoch in range(1, 1 + epochs):
    loss = train(model, data, train_idx, optimizer)
    result = test(model, data, split_idx, evaluator)

    if epoch % log_steps == 0:
        train_acc, valid_acc, test_acc = result
        print(f'Epoch: {epoch:02d}, '
              f'Loss: {loss:.4f}, '
              f'Train: {100 * train_acc:.2f}%, '
              f'Valid: {100 * valid_acc:.2f}% '
              f'Test: {100 * test_acc:.2f}%')

Epoch: 25, Loss: 1.4626, Train: 53.29%, Valid: 55.16% Test: 55.52%
Epoch: 50, Loss: 1.2140, Train: 66.69%, Valid: 67.30% Test: 66.58%
Epoch: 75, Loss: 1.1014, Train: 70.05%, Valid: 69.71% Test: 68.76%
Epoch: 100, Loss: 1.0423, Train: 71.48%, Valid: 70.50% Test: 69.59%
Epoch: 125, Loss: 1.0004, Train: 72.54%, Valid: 71.25% Test: 70.48%
Epoch: 150, Loss: 0.9666, Train: 73.23%, Valid: 71.68% Test: 70.93%
Epoch: 175, Loss: 0.9392, Train: 73.84%, Valid: 71.66% Test: 70.82%
Epoch: 200, Loss: 0.9159, Train: 74.46%, Valid: 71.94% Test: 70.91%
Epoch: 225, Loss: 0.8958, Train: 74.99%, Valid: 72.25% Test: 71.55%
Epoch: 250, Loss: 0.8782, Train: 75.49%, Valid: 72.20% Test: 71.21%
Epoch: 275, Loss: 0.8567, Train: 75.90%, Valid: 72.25% Test: 71.41%
Epoch: 300, Loss: 0.8446, Train: 76.37%, Valid: 72.42% Test: 71.52%
Epoch: 325, Loss: 0.8257, Train: 76.60%, Valid: 72.54% Test: 71.59%
Epoch: 350, Loss: 0.8138, Train: 76.83%, Valid: 72.75% Test: 72.09%
Epoch: 375, Loss: 0.8039, Train: 77.39%, Valid: 72.